In [1]:
import torch
from torch import nn
# from Teacher import Teacher_Model
from Teacher import Teacher_Model
import numpy as np
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch.nn.functional as F
from torch import optim
import os
# from Utils_tool import EarlyStopping
# from Utils_tool_1 import load_data_drink
# from Utils_tool_1 import load_data_exercise
# from Utils_tool_1 import load_data_mimic_ii
import sys
# from transformers import AdamW
import torch.optim as optim
from datetime import datetime

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
from torch.utils.data import Dataset


class MyDataset(Dataset):

    def __init__(self, x_data, y_data):
        self.x_data = x_data
        self.y_data = y_data

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, index):
        return torch.Tensor(np.array(self.x_data[index])), torch.Tensor(
            np.array(self.y_data[index]))


In [4]:
import plotly.graph_objects as go


def visualization(y1):
    fig = go.Figure()

    fig.add_trace(
        go.Scatter(y=y1,
                   mode="lines",
                   line=dict(color="blue"),
                   name="predict value"))
    fig.update_layout()
    fig.show()


In [5]:
import numpy as np
import h5py


def load_data_pulseDB(idx):
    x_tol_arr = []
    y_tol_arr = []
    for i in range(1, 11):
        # file_name = '/home/yonghu/data_yonghu/code_python/PulseDB_Ten_0728/Group%02d_in_out_split.h5' % i
        file_name = '/home/yonghu/data_yonghu/code_python/VitalDB_dataset/VitalDB_Ten_0927/Group%02d_in_out_split.h5' % i
        print(file_name)
        with h5py.File(file_name, "r") as file:
            for dataset_name in file:
                if dataset_name == 'In_Signals':
                    In_Signals = file[dataset_name][()][:, :, ::2]
                    print(In_Signals.shape)
                    x_tol_arr.append(In_Signals)
                elif dataset_name == 'Out_Signals':
                    Out_Signals = file[dataset_name][()][:, :, ::2]
                    print(Out_Signals.shape)
                    y_tol_arr.append(Out_Signals)

    x_train_load_data = x_tol_arr[3:10] + x_tol_arr[0:1]
    x_val_load_data = x_tol_arr[2:3]
    x_test_load_data = x_tol_arr[1:2]
    x_train_load_data = np.vstack(x_train_load_data)
    x_val_load_data = np.vstack(x_val_load_data)
    x_test_load_data = np.vstack(x_test_load_data)
    print(x_train_load_data.shape)
    print(x_val_load_data.shape)
    print(x_test_load_data.shape)
    y_train_load_data = y_tol_arr[3:10] + y_tol_arr[0:1]
    y_val_load_data = y_tol_arr[2:3]
    y_test_load_data = y_tol_arr[1:2]
    y_train_load_data = np.vstack(y_train_load_data)
    y_val_load_data = np.vstack(y_val_load_data)
    y_test_load_data = np.vstack(y_test_load_data)
    print(y_train_load_data.shape)
    print(y_val_load_data.shape)
    print(y_test_load_data.shape)

    # y_test_load_data = y_test_load_data / 300

    # x_train_load_data = x_train_load_data[0:1000]
    # y_train_load_data = y_train_load_data[0:1000]
    # x_val_load_data = x_val_load_data[0:1000]
    # y_val_load_data = y_val_load_data[0:1000]
    # x_test_load_data = x_test_load_data[0:1000]
    # y_test_load_data = y_test_load_data[0:1000]

    x_train_data = torch.tensor(x_train_load_data)
    y_train_data = torch.tensor(y_train_load_data)
    x_val_data = torch.tensor(x_val_load_data)
    y_val_data = torch.tensor(y_val_load_data)
    x_test_data = torch.tensor(x_test_load_data)
    y_test_data = torch.tensor(y_test_load_data)

    train_data_set = MyDataset(x_train_data, y_train_data)
    val_data_set = MyDataset(x_val_data, y_val_data)
    test_data_set = MyDataset(x_test_data, y_test_data)

    return train_data_set, val_data_set, test_data_set


In [6]:
train_data_set, val_data_set, test_data_set = load_data_pulseDB("01")
import os
def create_folder_if_not_exists(folder_path):
    # 使用 os.path.exists() 判断文件夹是否存在
    if not os.path.exists(folder_path):
        # 如果文件夹不存在，则创建新的文件夹
        os.mkdir(folder_path)
        print(f"Folder '{folder_path}' created.\n")
    else:
        print(f"Folder '{folder_path}' already exists.\n")
        
dir_name = "VitalDB_DataLoader_tol/Folder02/"
create_folder_if_not_exists(dir_name)
torch.save(train_data_set, dir_name+"train_dataset.pt")
torch.save(val_data_set, dir_name+"val_dataset.pt")
torch.save(test_data_set, dir_name+"test_dataset.pt")

/home/yonghu/data_yonghu/code_python/VitalDB_dataset/VitalDB_Ten_0927/Group01_in_out_split.h5
(125730, 4, 625)
(125730, 1, 625)
/home/yonghu/data_yonghu/code_python/VitalDB_dataset/VitalDB_Ten_0927/Group02_in_out_split.h5
(125779, 4, 625)
(125779, 1, 625)
/home/yonghu/data_yonghu/code_python/VitalDB_dataset/VitalDB_Ten_0927/Group03_in_out_split.h5
(125779, 4, 625)
(125779, 1, 625)
/home/yonghu/data_yonghu/code_python/VitalDB_dataset/VitalDB_Ten_0927/Group04_in_out_split.h5
(125779, 4, 625)
(125779, 1, 625)
/home/yonghu/data_yonghu/code_python/VitalDB_dataset/VitalDB_Ten_0927/Group05_in_out_split.h5
(125659, 4, 625)
(125659, 1, 625)
/home/yonghu/data_yonghu/code_python/VitalDB_dataset/VitalDB_Ten_0927/Group06_in_out_split.h5
(125659, 4, 625)
(125659, 1, 625)
/home/yonghu/data_yonghu/code_python/VitalDB_dataset/VitalDB_Ten_0927/Group07_in_out_split.h5
(125659, 4, 625)
(125659, 1, 625)
/home/yonghu/data_yonghu/code_python/VitalDB_dataset/VitalDB_Ten_0927/Group08_in_out_split.h5
(125659, 4

In [ ]:
train_loader = DataLoader(train_data_set, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data_set, batch_size=64, shuffle=False)
test_loader = DataLoader(test_data_set, batch_size=64, shuffle=False)
print(len(train_loader), len(val_loader), len(test_loader))

In [ ]:
def train_model(model, train_loader, val_loader, optimizer, criterion=nn.L1Loss(), num_epochs=100, patience=7,
                save_path=None):
    best_val_loss = float('inf')
    no_improvement = 0

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0

        train_loader_tqdm = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}")
        for id, (inputs, targets) in enumerate(train_loader_tqdm):
            inputs = inputs.to(device)
            targets = targets.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

            train_loader_tqdm.set_postfix(loss=f"{total_loss / (id + 1):.4f}")
        avg_train_loss = total_loss / len(train_loader)

        model.eval()
        total_loss = 0.0
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, targets)
                total_loss += loss.item()

        avg_val_loss = total_loss / len(val_loader)
        print(
            f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {avg_train_loss:.6f}, Validation Loss: {avg_val_loss:.6f}, Best Loss: {best_val_loss:.6f}")

        if avg_val_loss < best_val_loss:
            print(
                f'Validation loss decreased ({best_val_loss:.6f} --> {avg_val_loss:.6f}).  Saving model ...'
            )
            best_val_loss = avg_val_loss
            no_improvement = 0

            if save_path is not None:
                current_datetime = datetime.now()
                current_datetime_str = current_datetime.strftime('_%Y_%m_%d_%H:%M:%S')
                path_name = save_path + current_datetime_str + ".pth"
                torch.save(model.state_dict(), path_name)
                best_model = model
                print(f"Saved the best model to '{path_name}'.")
            else:
                print("Not Save!!!!")

        else:
            no_improvement += 1
            print(
                f'EarlyStopping counter: {no_improvement} out of {patience}'
            )

        if no_improvement >= patience:
            print("Early Stopping! Training stopped.")
            break

    print("Training completed!")

In [ ]:
# 测试模型
def test_model(model, test_loader):
    model.eval()
    criterion = nn.L1Loss()
    total_loss = 0.0

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss += loss.item()

    print(f"Test Loss: {total_loss / len(test_loader)}")

In [ ]:
weight_path = "weight_model/UtransBPNet_VitalDB_Fold02_"
model = Teacher_Model(4, 128, 7, 3).to(device)

loss_fun = nn.L1Loss()
# opt = optim.Adam(model.parameters(), lr=1e-3)
opt = optim.AdamW(model.parameters(), lr=1e-5, weight_decay=0.1)


best_model = ""

train_model(model, train_loader=train_loader, val_loader=val_loader, optimizer=opt, criterion=loss_fun, num_epochs=100,
            patience=10, save_path=weight_path)

In [ ]:
test_model(model=best_model.to(device), test_loader=test_loader)

In [ ]:
# # 测试
# import plotly.graph_objects as go


# def visualization(y1, y2, mae):
#     ME = np.mean(y1 - y2)
#     SD = np.std(y1 - y2)
#     fig = go.Figure()

#     fig.add_trace(
#         go.Scatter(y=y1[0:12500],
#                    mode="lines",
#                    line=dict(color="blue"),
#                    name="predict value"))
#     fig.add_trace(
#         go.Scatter(y=y2[0:12500],
#                    mode="lines",
#                    line=dict(color="red"),
#                    name="actual value"))

#     tit_name = "mae = " + str(mae) + "      " + \
#                "me = " + str(ME) + "     " + "SD = " + str(SD)
#     fig.update_layout(title=tit_name)
#     fig.show()

# # visualization(ture_data, pred_data, mae_val)


In [ ]:
# model = Teacher_Model(4, 128, 7, 3)
# model.load_state_dict(torch.load(weight_path))

# y_true = np.array([])
# out = np.array([])
# with torch.no_grad():
#     model.eval()
#     for z, (x, y) in enumerate((test_loader)):
#         input_data = x
#         y = y
#         temp = model(input_data)
#         out = np.append(out, temp.detach().cpu().numpy().flatten())
#         y_true = np.append(y_true, y.detach().cpu().numpy().flatten())

#     out = out
#     y_true = y_true
#     mae = np.mean(np.abs(out - y_true))
#     print("MAE = ", mae)

# visualization(y_true, out, mae)